In [ ]:
import sys
print(sys.executable)

!{sys.executable} -m pip install opencv-python mediapipe numpy --upgrade

/opt/homebrew/opt/python@3.11/bin/python3.11
  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached mediapipe-0.10.21-cp311-cp311-macosx_11_0_universal2.whl.metadata (9.9 kB)
  Using cached numpy-2.3.5-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached numpy-2.2.6-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached jax-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.8.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (1.3 kB)
  Using cached matplotlib-3.10.7-cp311-cp311-macosx_11_0_arm64.whl.metadata (11 kB)
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
  Using cached mediapipe-0.10.20-cp311-cp311-maco

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time # FPS hesaplamak için

print("Kütüphaneler başarıyla içe aktarıldı.")

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Kütüphaneler başarıyla içe aktarıldı.


In [ ]:
# MediaPipe Hands modelini başlatma
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False, # Video akışı için False
    max_num_hands=1,         # Maksimum 1 el takip edilsin
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# MediaPipe çizim araçlarını başlatma
mp_drawing = mp.solutions.drawing_utils

finger_tip_ids = [8, 12, 16, 20]

finger_colors = {
    8: (255, 0, 0),    # İşaret Parmağı: Mavi
    12: (0, 255, 0),   # Orta Parmak: Yeşil
    16: (0, 0, 255),   # Yüzük Parmağı: Kırmızı
    20: (255, 0, 255)  # Serçe Parmak: Mor
}

finger_tracks = {id: [] for id in finger_tip_ids}

MAX_TRACE_POINTS = 50

I0000 00:00:1763719999.797903  517050 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M3


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1763719999.812555  518994 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763719999.825277  518994 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [6]:
# Video yakalama objesi
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("HATA: Kamera açılamadı.")
    # cv2.VideoCapture(0) yerine bazen 1 veya farklı bir kamera indeksi gerekebilir.
    exit()

pTime = 0

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Kamera akışı alınamıyor, çıkılıyor...")
        break

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for id in finger_tip_ids:
                landmark = hand_landmarks.landmark[id]
                h, w, c = image.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                finger_tracks[id].append((cx, cy))
                if len(finger_tracks[id]) > MAX_TRACE_POINTS:
                    finger_tracks[id].pop(0)
                cv2.circle(image, (cx, cy), 5, finger_colors[id], cv2.FILLED)
                
            for id, track_points in finger_tracks.items():
                if len(track_points) >= 2:
                    color = finger_colors[id]
                    for i in range(1, len(track_points)):
                        pt1 = track_points[i-1]
                        pt2 = track_points[i]
                        thickness = int(np.interp(i, [1, MAX_TRACE_POINTS], [1, 5])) 
                        cv2.line(image, pt1, pt2, color, thickness)

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(image, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)
    
    # !!! YEREL ORTAM GÖSTERİMİ !!!
    cv2.imshow('El Takibi ile Coklu Renkli Cizgi', image)

    # 'q' tuşuna basılırsa döngüden çık
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Temizlik işlemleri
cap.release()
cv2.destroyAllWindows()

/opt/homebrew/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 